In [1]:
import numpy as np 
from qutip import *
from qutip.qip.circuit import *
from qutip.qip.operations import *
from qutip import tensor
import random 

# bit flip simulation trial #1 
up = basis(2,0)    # spin up
down = basis(2, 1)    # spin down

# bit setup; state before error 
psi = (up + down).unit()    
init_state1 = tensor(psi, up , up)   # a|0> + b|1>
#print(init_state1)

#encoding q1 and q2 
q1 = cnot(N = 3, control = 0, target = 1)
q2 = cnot(N = 3, control = 0, target = 2)
init_state2 = q2*q1*init_state1  # a|000> + b|111>
#print(init_state2)

#flip circuit into the |+>|-> domain
h01 = snot(N = 3, target = 0)
h11 = snot(N = 3, target = 1)
h21 = snot(N = 3, target = 2)
init_state3 = h21*h11*h01*init_state2  # a|+++> + b|---> 

#syndrome meas. 
A = tensor(sigmaz(), sigmaz(), qeye(2))  #ZZI
B = tensor(qeye(2), sigmaz(), sigmaz())  #IZZ

#error 
# bit flip determination
q0_flip = tensor(sigmaz(), qeye(2), qeye(2)) # flip in the first qubit 
q1_flip = tensor(qeye(2), sigmaz(), qeye(2)) # flip in the second qubit 
q2_flip = tensor(qeye(2), qeye(2), sigmaz()) # flip in the third qubit 
states = [q0_flip, q1_flip, q2_flip]
c = random.choice(states)
err_state = c*init_state3
print("Error in |+>|-> domain: \n{}\n".format(err_state))

#flip back into the a|0> + b|1> domain
h02 = snot(N = 3, target = 0)
h12 = snot(N = 3, target = 1)
h22 = snot(N = 3, target = 2)
init_state4 = h22*h12*h02*err_state   #state after error 
print('Error flipped into the |0>|1> domain: ')

a = 1 if (A*init_state4) == init_state4 else -1 
b = 1 if (B*init_state4) == init_state4 else -1  
print('syndrome meas: {},{}'.format(a, b))
print()

# error detection & correction
if a ==1 and b ==1: 
    print("No flip: \n {}".format(init_state4))
elif a == -1 and b == 1:
    print("q0 is flipped: \n {}\n".format(init_state4))
    err_corr = tensor(sigmax(), qeye(2), qeye(2))*init_state4
    print("Corrected to \n {}".format(err_corr))
elif a == 1 and b == -1:
    print("q1 is flipped: \n {}\n".format(init_state4))
    err_corr = tensor(qeye(2), qeye(2), sigmax())*init_state4
    print("Corrected to \n {}".format(err_corr))
elif a == -1 and b == -1:
    print("q2 is flipped: \n {}\n".format(init_state4))
    err_corr = tensor(qeye(2), sigmax(), qeye(2))*init_state4
    print("Corrected to \n {}".format(err_corr))
    

Error in |+>|-> domain: 
Quantum object: dims = [[2, 2, 2], [1, 1, 1]], shape = (8, 1), type = ket
Qobj data =
[[ 0.5]
 [ 0. ]
 [ 0. ]
 [-0.5]
 [ 0. ]
 [ 0.5]
 [-0.5]
 [ 0. ]]

Error flipped into the |0>|1> domain: 
syndrome meas: -1,-1

q2 is flipped: 
 Quantum object: dims = [[2, 2, 2], [1, 1, 1]], shape = (8, 1), type = ket
Qobj data =
[[0.        ]
 [0.        ]
 [0.70710678]
 [0.        ]
 [0.        ]
 [0.70710678]
 [0.        ]
 [0.        ]]

Corrected to 
 Quantum object: dims = [[2, 2, 2], [1, 1, 1]], shape = (8, 1), type = ket
Qobj data =
[[0.70710678]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.70710678]]


In [48]:
# print psi after cnot gates 
a = ket2dm(init_state1)
# with error 
b = ket2dm(init_state4)
print(a, b)

Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[0.5 0.  0.  0.  0.5 0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.5 0.  0.  0.  0.5 0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]] Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.5 0.  0.  0.  0.  0.5 0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.5 0.  0.  0.  0.  0.5 0. ]
 [0.  0.  0.  0.  0.  0.  0.  0. ]]


In [13]:
#Z0, Z1, = ket2dm(basis(2, 0)), ket2dm(basis(2, 1))

#PZ1 = [tensor(Z0, identity(2)), tensor(Z1, identity(2))]
#PZ2 = [tensor(identity(2), Z0), tensor(identity(2), Z1)]
#collapsed_states, probabilities = measurement_statistics(c , PZ2)

#print(collapsed_states)

In [ ]:
from qutip.measurement import measure, measurement_statistics